# Decorating Classes
## Python is a dynamic language, that means we can actually modify things in object and classes during run time while the codes are executing. We can go ahead that was previously defined. 
## monkey patching in python

In [2]:
from fractions import Fraction

In [3]:
f = Fraction(2,3)

In [4]:
f.denominator

3

In [5]:
f.numerator

2

In [6]:
f.speak()

AttributeError: 'Fraction' object has no attribute 'speak'

In [7]:
Fraction.speak = 100 # You can even modify the standard library to change the methods

In [9]:
f.speak # this is an attribute now. 

100

In [10]:
Fraction.speak = lambda self, message: 'Fraction says: {0}'.format(message)

In [11]:
f.speak('This is a late parrot')

'Fraction says: This is a late parrot'

In [12]:
f2 = Fraction(10,5)

In [13]:
f2.speak('This parrot is no more.')

'Fraction says: This parrot is no more.'

In [14]:
# adding attributes in runtime

In [15]:
# modifying classes when it's running

In [16]:
Fraction.is_integral = lambda self: self.denominator == 1

In [17]:
f1 = Fraction(2,3)

In [18]:
f2 = Fraction(64, 8)

In [19]:
f1

Fraction(2, 3)

In [20]:
f2

Fraction(8, 1)

In [21]:
f1.is_integral()

False

In [22]:
f2.is_integral()

True

# Now we monkey patch the class and use a function to decorate/modify the fraction class

In [37]:
def dec_speak(cls): 
    cls.speak = lambda self, message: '{0} says: {1}'.format(self.__class__.__name__, message)
    
    return cls

In [38]:
Fraction = dec_speak(Fraction) # dec_speak is decorating Fraction

In [39]:
f1 = Fraction(2,3)

In [40]:
f1.speak('hello')

'Fraction says: hello'

In [41]:
class Person: 
    pass

In [42]:
Person = dec_speak(Person)

In [43]:
p = Person()

In [44]:
p.speak('this works')

'Person says: this works'

In [45]:
from datetime import datetime, timezone

In [46]:
def debug_inf(cls):
    def info(self): 
        results = []
        results.append('time: {0}'.format(datetime.now(timezone.utc)))
        results.append('Class: {0}'.format(self.__class__.__name__))
        results.append('id: {0}'.format(hex(id(self))))
        for k, v in vars(self).items():  # vars returns all the properties of that object itself
            results.append('{0}: {1}'.format(k,v))
        return results  #not a closure so far, as there is no free variable. It's just nesting inside
    
    cls.debug = info

In [50]:
def info(self): 
    results = []
    results.append('time: {0}'.format(datetime.now(timezone.utc)))
    results.append('Class: {0}'.format(self.__class__.__name__))
    results.append('id: {0}'.format(hex(id(self))))
    for k, v in vars(self).items():  # vars returns all the properties of that object itself
        results.append('{0}: {1}'.format(k,v))
    return results  #not a closure so far, as there is no free variable. It's just nesting inside

def debug_info(cls): 
    cls.debug = info
    return cls

In [61]:
@debug_info
class Person: 
    def __init__(self, name, birth_year): 
        self.name = name
        self.birth_year = birth_year
        
    def say_hi(): 
        return 'Hello there!'

In [62]:
p = Person('John', 1939)

In [63]:
p.debug()

['time: 2021-02-15 18:01:54.270721+00:00',
 'Class: Person',
 'id: 0x114d44b20',
 'name: John',
 'birth_year: 1939']

In [64]:
del Person
del p

In [65]:
@debug_info
class Person: 
    def __init__(self, name): 
        self.name = name
        

In [66]:
p = Person()

TypeError: __init__() missing 1 required positional argument: 'name'

In [68]:
Person = debug_info(Person)

In [69]:
p = Person('John')

In [70]:
@debug_info
class Person: 
    def __init__(self, name): 
        self.name = name

In [72]:
p.debug() # can be resued for other classes too. 

['time: 2021-02-15 18:15:24.622917+00:00',
 'Class: Person',
 'id: 0x114d44e20',
 'name: John']

In [76]:
@debug_info
class Automobile: 
    def __init__(self, make, model, year, top_speed): 
        self.make = make
        self.model = model
        self.year = year
        self.top_speed = top_speed
        self._speed = 0 
        
    @property
    def speed(self): 
        return self._speed
    
    @speed.setter 
    def speed(self, new_speed): 
        if new_speed > self.top_speed: 
            raise ValueError('Speed cannot exceed top_speed.')
        else: 
            self._speed = new_speed

In [77]:
favorite = Automobile('Ford', 'Model T', 1908, 45)

In [78]:
favorite.debug()

['time: 2021-02-15 18:27:01.199344+00:00',
 'Class: Automobile',
 'id: 0x114d449d0',
 'make: Ford',
 'model: Model T',
 'year: 1908',
 'top_speed: 45',
 '_speed: 0']

In [80]:
favorite.speed = 100

ValueError: Speed cannot exceed top_speed.

In [81]:
favorite.speed = 40

In [82]:
favorite.debug()

['time: 2021-02-15 18:27:31.756224+00:00',
 'Class: Automobile',
 'id: 0x114d449d0',
 'make: Ford',
 'model: Model T',
 'year: 1908',
 'top_speed: 45',
 '_speed: 40']

In [83]:
from math import sqrt

In [84]:
class Point: 
    def __init__(self, x, y): 
        self.x = x
        self.y = y
        
    def __abs__(self): # will be able to call the abosolute function on a Point object 
        return sqrt(self.x ** 2 + self.y ** 2)
    
    def __repr__(self): 
        return 'Point({0}, {1})'.format(self.x, self.y)
    
        

In [85]:
p1, p2, p3 = Point(2,3), Point(2,3), Point(0,0)

In [86]:
abs(p1)

3.605551275463989

In [87]:
p1

Point(2, 3)

In [88]:
p1 is p2

False

In [89]:
p2 is p3

False

In [90]:
p1 == p2

False

In [91]:
class Point: 
    def __init__(self, x, y): 
        self.x = x
        self.y = y
        
    def __abs__(self): # will be able to call the abosolute function on a Point object 
        return sqrt(self.x ** 2 + self.y ** 2)
    
    def __repr__(self): 
        return 'Point({0}, {1})'.format(self.x, self.y)
    
    def __eq__(self, other): 
        if isinstance(other, Point): 
            return self.x == other.x and self.y == other.y 
        else: 
            return False 
        

In [92]:
p1, p2, p3 = Point(2,3), Point(2,3), Point(0,0)

In [93]:
p1 is p2

False

In [94]:
p1 == p2

True

In [96]:
p1 < p2 # not supported so far, and we will implement a new method to do this. 

TypeError: '<' not supported between instances of 'Point' and 'Point'

In [97]:
class Point: 
    def __init__(self, x, y): 
        self.x = x
        self.y = y
        
    def __abs__(self): # will be able to call the abosolute function on a Point object 
        return sqrt(self.x ** 2 + self.y ** 2)
    
    def __repr__(self): 
        return 'Point({0}, {1})'.format(self.x, self.y)
    
    def __eq__(self, other): 
        if isinstance(other, Point): 
            return self.x == other.x and self.y == other.y 
        else: 
            return False 
        
    def __lt__(self, other): 
        if isinstance(other, Point): 
            return abs(self) < abs(other)
        else: 
            return NotImplemented 

In [98]:
p1, p2, p3 = Point(2,3), Point(2,3), Point(0,0)

In [99]:
p1 == p2

True

In [100]:
p3 < p1

True

In [101]:
p4 = Point(100, 100)

In [102]:
p4 < p1

False

In [103]:
p4 > p1

True

In [104]:
p1 < p4

True

In [105]:
p1 <= p4

TypeError: '<=' not supported between instances of 'Point' and 'Point'

In [106]:
class Point: 
    def __init__(self, x, y): 
        self.x = x
        self.y = y
        
    def __abs__(self): # will be able to call the abosolute function on a Point object 
        return sqrt(self.x ** 2 + self.y ** 2)
    
    def __repr__(self): 
        return 'Point({0}, {1})'.format(self.x, self.y)
    
    def __eq__(self, other): 
        if isinstance(other, Point): 
            return self.x == other.x and self.y == other.y 
        else: 
            return False 
        
    def __lt__(self, other): 
        if isinstance(other, Point): 
            return abs(self) < abs(other)
        else: 
            return NotImplemented 
        
#     def __le__(self, other):     These are not neccessary, as you can build them up in __eq__ and __lt__ using Decorator
#         pass                     Monkey Patch
    
#     def __gt__(self, other): 
#         pass 
    
#     def __ge__(self, other): 
#         pass 
    
#     def __ne__(self, other): 
#         pass 

## a <= b iff a < b or a == b 
## a >b iff not (a<b) and a != b
## a>=b iff not(a<b) 

In [116]:
def complete_ordering(cls): 
    if '__eq__' in dir(cls) and '__lt__' in dir(cls): 
        cls.__le__ = lambda self, other: self < other or self == other
        cls.__gt__ = lambda self, other: not (self < other) and not (self == other)
        cls.__ge__ = lambda self, other: not (self < other)
    return cls

In [117]:
@complete_ordering
class Point: 
    def __init__(self, x, y): 
        self.x = x
        self.y = y
        
    def __abs__(self): # will be able to call the abosolute function on a Point object 
        return sqrt(self.x ** 2 + self.y ** 2)
    
    def __repr__(self): 
        return 'Point({0}, {1})'.format(self.x, self.y)
    
    def __eq__(self, other): 
        if isinstance(other, Point): 
            return self.x == other.x and self.y == other.y 
        else: 
            return False 
        
    def __lt__(self, other): 
        if isinstance(other, Point): 
            return abs(self) < abs(other)
        else: 
            return NotImplemented 

In [118]:
p1, p2, p3, p4 = Point(2,3), Point(2,3), Point(0,0), Point(100, 200)

In [119]:
p1 <= p4

True

In [120]:
p2 >= p2

True

In [121]:
p1 != p2

False

In [126]:
from functools import total_ordering  # from standard library. It will figure out which comparison is missing and 
# will fill out the rest

In [127]:
@total_ordering
class Point: 
    def __init__(self, x, y): 
        self.x = x
        self.y = y
        
    def __abs__(self): # will be able to call the abosolute function on a Point object 
        return sqrt(self.x ** 2 + self.y ** 2)
    
    def __repr__(self): 
        return 'Point({0}, {1})'.format(self.x, self.y)
    
    def __eq__(self, other): 
        if isinstance(other, Point): 
            return self.x == other.x and self.y == other.y 
        else: 
            return False 
        
    def __lt__(self, other): 
        if isinstance(other, Point): 
            return abs(self) < abs(other)
        else: 
            return NotImplemented 

In [128]:
p1, p2, p3, p4 = Point(2,3), Point(2,3), Point(0,0), Point(100, 200)


In [129]:
p1 <= p2 

True

In [130]:
p1 >= p2

True

In [131]:
# complete_ordering is what we did to replicate total_ordering

In [132]:
@total_ordering
class Point: 
    def __init__(self, x, y): 
        self.x = x
        self.y = y
        
    def __abs__(self): # will be able to call the abosolute function on a Point object 
        return sqrt(self.x ** 2 + self.y ** 2)
    
    def __repr__(self): 
        return 'Point({0}, {1})'.format(self.x, self.y)
    
    def __eq__(self, other): 
        if isinstance(other, Point): 
            return self.x == other.x and self.y == other.y 
        else: 
            return False 
        
    def __gt__(self, other):      # we changed this part, and total_ordering just figured it out automatically 
        if isinstance(other, Point): 
            return abs(self) > abs(other)
        else: 
            return NotImplemented 

In [133]:
p1, p2, p3, p4 = Point(2,3), Point(2,3), Point(0,0), Point(100, 200)


In [134]:
p4 > p1

True

In [136]:
p2 <= p3

False